In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [3]:
def poisson(lambdain,n) :
    return ((pow(lambdain,n))*(math.exp(-lambdain)))/(math.factorial(n))

In [4]:
maxCars=20
maxTrans=5
transCost=-2
rentCost=10
lambdaReq1=3
lambdaReq2=4
lambdaRet1=3
lambdaRet2=2
gamma=0.9

In [5]:
states=[]
for i in range(maxCars+1) :
    for j in range(maxCars+1) :
        states.append([i,j])

policy = np.zeros((maxCars + 1, maxCars + 1))
stateRet = np.zeros((maxCars + 1, maxCars + 1))
newStateRet = np.zeros((maxCars + 1, maxCars + 1))
actions=[-5,-4,-3,-2,-1,0,1,2,3,4,5]
needImprovePolicy = False

In [6]:
def expectedReturn(state,action,stateRet) :

    returns=0.0

    cars1Start=min(state[0]-action , maxCars)
    cars2Start=min(state[1]+action , maxCars)

    for rentals1 in range(0,maxCars + 1) :
        for rentals2 in range(0,maxCars + 1) :

            rentalProb=poisson(lambdaReq1,rentals1)*poisson(lambdaReq2,rentals2)
            totalRentals1=min(cars1Start,rentals1)
            totalRentals2=min(cars2Start,rentals2)
            rewards=(totalRentals1+totalRentals2)*rentCost

            for returns1 in range(0,maxCars+1) :
                for returns2 in range(0,maxCars+1) :

                    returnProb=poisson(lambdaRet1,returns1)*poisson(lambdaRet2,returns2)
                    cars1End=int(min(cars1Start-totalRentals1+returns1 , maxCars))
                    cars2End=int(min(cars2Start-totalRentals2+returns2 , maxCars))

                    totalProb=rentalProb*returnProb
                    returns=returns+totalProb*(rewards+gamma*stateRet[cars1End,cars2End])
    
    return returns

In [7]:

# while True :
#     while needImprovePolicy==False :
#         for i,j in states :
#             newStateRet[i,j]=expectedReturn([i,j],policy[i,j],stateRet)

#         if np.sum(np.absolute(newStateRet - stateRet)) < 1e-4:
#             stateRet = newStateRet.copy()
#             needImprovePolicy = True
#             break

# if needImprovePolicy==True :
#     tmpPolicy=np.zeros((maxCars+1,maxCars+1))
#     for i,j in states :
#         actionReturns=[]
#         for action in actions :
#             if ((action >= 0 and i >= action) or (action < 0 and j >= np.absolute(action))):
#                         actionReturns.append(expectedReturn([i, j], action, stateRet))
#             else : actionReturns.append(-float('inf'))
#         tmpPolicy[i,j]=actions[np.argmax(actionReturns)]

#     policyChanges = np.sum(tmpPolicy != policy)
#     if policyChanges == 0:
#         policy = tmpPolicy
#         break
#     policy = tmpPolicy
#     needImprovePolicy = False

In [ ]:
iterations = 0
_, axes = plt.subplots(2, 3, figsize=(40, 20))
plt.subplots_adjust(wspace=0.1, hspace=0.2)
axes = axes.flatten()
while True:
    fig = sns.heatmap(np.flipud(policy), cmap="YlGnBu", ax=axes[iterations])
    fig.set_ylabel('# cars at first location', fontsize=30)
    fig.set_yticks(list(reversed(range(maxCars + 1))))
    fig.set_xlabel('# cars at second location', fontsize=30)
    fig.set_title('policy {}'.format(iterations), fontsize=30)

    # policy evaluation (in-place)
    while True:
        old_stateRet = stateRet.copy()
        for i in range(maxCars + 1):
            for j in range(maxCars + 1):
                new_state_stateRet = expectedReturn([i, j], policy[i, j], stateRet)
                stateRet[i, j] = new_state_stateRet
        max_stateRet_change = abs(old_stateRet - stateRet).max()
        print('max stateRet change {}'.format(max_stateRet_change))
        if max_stateRet_change < 1e-4:
            break

    # policy improvement
    policy_stable = True
    for i in range(maxCars + 1):
        for j in range(maxCars + 1):
            old_action = policy[i, j]
            action_returns = []
            for action in actions:
                if (0 <= action <= i) or (-j <= action <= 0):
                        action_returns.append(expectedReturn([i, j], action, stateRet))
                else:
                    action_returns.append(-np.inf)
            new_action = actions[np.argmax(action_returns)]
            policy[i, j] = new_action
            if policy_stable and old_action != new_action:
                policy_stable = False
    print('policy stable {}'.format(policy_stable))
    if policy_stable:
            fig = sns.heatmap(np.flipud(stateRet), cmap="YlGnBu", ax=axes[-1])
            fig.set_ylabel('# cars at first location', fontsize=30)
            fig.set_yticks(list(reversed(range(maxCars + 1))))
            fig.set_xlabel('# cars at second location', fontsize=30)
            fig.set_title('optimal stateRet', fontsize=30)
            break

    iterations += 1

    plt.savefig('../images/figure_4_2.png')
    plt.close()

max stateRet change 136.16756138575548
max stateRet change 91.93257483528754
max stateRet change 64.58575734351558
max stateRet change 46.05623790157284
max stateRet change 33.83219569628881
max stateRet change 27.181405826162802
max stateRet change 21.944123810316853
max stateRet change 17.79529439704828
max stateRet change 15.09536231976989
max stateRet change 12.869205890681314
max stateRet change 10.923156653104229
max stateRet change 9.232320081170712
max stateRet change 7.773300563878763
max stateRet change 6.52260727574901
max stateRet change 5.456870809069983
max stateRet change 4.5534793415114905
max stateRet change 3.7911603860645187
max stateRet change 3.1503786246415757
max stateRet change 2.6135450878664983
max stateRet change 2.165073944092626
max stateRet change 1.79133092185765
max stateRet change 1.4805124843274484
max stateRet change 1.222486253541831
max stateRet change 1.008614618278898
max stateRet change 0.8315763133805945
max stateRet change 0.685195296956465
max